In [ ]:
# Cell 1
!pip install pandas numpy matplotlib seaborn scikit-learn vaderSentiment kaggle nbconvert reportlab statsmodels


In [1]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi

# 1. Set Kaggle API credentials (optional if you have ~/.kaggle/kaggle.json)
os.environ['KAGGLE_USERNAME'] = os.getenv('KAGGLE_USERNAME', 'your_username')
os.environ['KAGGLE_KEY'] = os.getenv('KAGGLE_KEY', 'your_api_key')

api = KaggleApi()
api.authenticate()  # authenticates using environment variables or ~/.kaggle/kaggle.json

# 2. Download and unzip the dataset
dataset_ref = "tmdb/tmdb-movie-metadata"
download_dir = "./data"                      # wherever you want the files
api.dataset_download_files(dataset_ref, path=download_dir, unzip=True)

# Quick sanity‑check: list the extracted CSVs
import glob, pathlib, textwrap, pprint
csv_files = glob.glob(str(pathlib.Path(download_dir) / "*.csv"))
print("Extracted CSV files:")
pprint.pprint(csv_files)


Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
Extracted CSV files:
['data\\tmdb_5000_credits.csv', 'data\\tmdb_5000_movies.csv']


In [2]:
#  Cell 4 – Load & preprocess the data
import pandas as pd
import json
import numpy as np

movies_path  = pathlib.Path(download_dir) / "tmdb_5000_movies.csv"

df = pd.read_csv(movies_path)

# ---- Minimal cleaning / feature engineering ----
#  Keep only rows that have the core numeric columns we need
df = df.dropna(subset=["budget", "revenue", "overview", "genres"])

#  Rename columns to match our later code
df = df.rename(columns={"revenue": "box_office",
                        "overview": "reviews"})

#  Create a simple primary‑genre column from the JSON‑encoded list
def extract_first_genre(genres_json):
    try:
        genres = json.loads(genres_json.replace("'", '"'))
        return genres[0]["name"] if genres else "Unknown"
    except Exception:
        return "Unknown"
df["genre"] = df["genres"].apply(extract_first_genre)

#  Parse release year
df["release_year"] = pd.to_datetime(df["release_date"], errors="coerce").dt.year

#  Profit and binary success flag
df["profit"]  = df["box_office"] - df["budget"]
df["success"] = (df["profit"] > 0).astype(int)

#  Handle remaining numeric missing values
num_cols = ["runtime", "budget"]
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# Drop rows with any still‑missing critical fields
df = df.dropna(subset=["runtime", "reviews", "genre", "release_year"])

print(f"Dataset ready Rows: {len(df):,} | Columns: {df.shape[1]}")
df.head(3)


Dataset ready Rows: 4,799 | Columns: 24


budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   

                                             reviews  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   
2  A cryptic message from Bond’s past sends him o...  107.376788   

                                production_companies  ...  \
0  [{"name": "Ingenious Film Partners", "id": 289...  ...   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...  ...   
2  [{"name": "Columbia Pictures", "id": 5}, {"nam...  ...   

                                    spoken_languages    status  \
0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   
1           [{"iso_639_1": "en", "name": "English"}]  Released   
2  [{"iso_639_1": "fr", "name": "Fran\u00e7ais"},...  Released   

                                          tagline  \
0                     Enter the World of Pandora.   
1  At the end of the world, the adventure begins.   
2                           A Plan No One Escapes   

                                      title vote_average vote_count  \
0                                    Avatar          7.2      11800   
1  Pirates of the Caribbean: At World's End          6.9       4500   
2                                   Spectre          6.3       4466   

       genre release_year      profit  success  
0     Action       2009.0  2550965087        1  
1  Adventure       2007.0   661000000        1  
2     Action       2015.0   635674609        1  

[3 rows x 24 columns]

In [3]:
# Cell 5 – Sentiment scores with VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

df["sentiment"] = df["reviews"].astype(str).apply(
    lambda txt: analyzer.polarity_scores(txt)["compound"]
)
df["sentiment_label"] = df["sentiment"].apply(
    lambda s: "positive" if s >= 0.05 else "negative" if s <= -0.05 else "neutral"
)

df[["title", "sentiment", "sentiment_label"]].head()


title  sentiment sentiment_label
0                                    Avatar    -0.3612        negative
1  Pirates of the Caribbean: At World's End    -0.3919        negative
2                                   Spectre    -0.8271        negative
3                     The Dark Knight Rises    -0.9136        negative
4                               John Carter    -0.7096        negative

In [4]:
#  Cell 6 – Train/test split & regression model (box‑office prediction)
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

feature_cols   = ["budget", "runtime", "sentiment", "genre"]
target_col     = "box_office"

X = df[feature_cols].copy()
y = df[target_col]

# Preprocess: one‑hot encode genre, keep numerics passthrough
preprocessor = ColumnTransformer(
    transformers=[("genre_enc", OneHotEncoder(handle_unknown="ignore"), ["genre"])],
    remainder="passthrough"
)

model = Pipeline(steps=[
    ("prep", preprocessor),
    ("reg",  LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [5]:
# Updated Cell 7 – Evaluate model with more metrics and export to PDF
from sklearn.metrics import (
    mean_squared_error, r2_score,
    mean_absolute_error, mean_absolute_percentage_error
)
from math import sqrt
from reportlab.pdfgen import canvas

# ---- Evaluation Metrics ----
mse   = mean_squared_error(y_test, y_pred)
rmse  = sqrt(mse)
mae   = mean_absolute_error(y_test, y_pred)
mape  = mean_absolute_percentage_error(y_test, y_pred)
r2    = r2_score(y_test, y_pred)

# Adjusted R²
n = len(y_test)
p = X_train.shape[1]  # number of features after transformation
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

# ---- PDF Generation ----
report_path = "model_evaluation_report.pdf"
c = canvas.Canvas(report_path)
c.setFont("Helvetica-Bold", 14)
c.drawString(72, 800, "Movie Success Prediction – Model Evaluation Report")

c.setFont("Helvetica", 12)
y_offset = 775
line_height = 18

metrics = [
    f"Total Observations: {len(df):,}",
    f"Test Set Size: {len(y_test):,}",
    f"",
    f"Mean Squared Error (MSE):     {mse:,.0f}",
    f"Root Mean Squared Error (RMSE): ${rmse:,.0f}",
    f"Mean Absolute Error (MAE):     ${mae:,.0f}",
    f"Mean Absolute Percentage Error (MAPE): {mape:.2%}",
    f"R² Score:                      {r2:.3f}",
    f"Adjusted R² Score:            {adj_r2:.3f}",
]

for line in metrics:
    c.drawString(72, y_offset, line)
    y_offset -= line_height

c.save()
print(f"📄 PDF saved  {report_path}")


📄 PDF saved  model_evaluation_report.pdf


In [6]:
#  Updated Cell – Save boxplots per genre
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Create output directory
visuals_dir = "visuals"
os.makedirs(visuals_dir, exist_ok=True)

# Plot sentiment for each genre as a boxplot
for genre in df["genre"].unique():
    genre_data = df[df["genre"] == genre]

    if genre_data.empty:
        continue  # skip if no data

    plt.figure(figsize=(6, 4))
    sns.boxplot(y=genre_data["sentiment"], color="lightblue")
    plt.title(f"Sentiment Distribution – {genre}", fontsize=12)
    plt.ylabel("VADER Sentiment Score")
    plt.ylim(-1, 1)
    plt.grid(True, axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()

    # Safe filename
    safe_genre = genre.replace("/", "_").replace(" ", "_")
    plot_path = os.path.join(visuals_dir, f"boxplot_sentiment_{safe_genre}.png")
    plt.savefig(plot_path, dpi=300)
    plt.close()

    print(f"Saved: {plot_path}")


Saved: visuals\boxplot_sentiment_Action.png
Saved: visuals\boxplot_sentiment_Adventure.png
Saved: visuals\boxplot_sentiment_Fantasy.png
Saved: visuals\boxplot_sentiment_Animation.png
Saved: visuals\boxplot_sentiment_Science_Fiction.png
Saved: visuals\boxplot_sentiment_Drama.png
Saved: visuals\boxplot_sentiment_Thriller.png
Saved: visuals\boxplot_sentiment_Family.png
Saved: visuals\boxplot_sentiment_Comedy.png
Saved: visuals\boxplot_sentiment_History.png
Saved: visuals\boxplot_sentiment_War.png
Saved: visuals\boxplot_sentiment_Western.png
Saved: visuals\boxplot_sentiment_Romance.png
Saved: visuals\boxplot_sentiment_Crime.png
Saved: visuals\boxplot_sentiment_Mystery.png
Saved: visuals\boxplot_sentiment_Horror.png
Saved: visuals\boxplot_sentiment_Documentary.png
Saved: visuals\boxplot_sentiment_Music.png
Saved: visuals\boxplot_sentiment_TV_Movie.png
Saved: visuals\boxplot_sentiment_Unknown.png
Saved: visuals\boxplot_sentiment_Foreign.png


In [8]:
# 📃 Cell 9 – Statsmodels OLS summary (predictive model summary)
import statsmodels.api as sm

# Encode categoricals manually for statsmodels
X_enc = pd.get_dummies(X, drop_first=True)
X_enc = sm.add_constant(X_enc)

ols_model = sm.OLS(y, X_enc).fit()
print(ols_model.summary())

# Persist summary to disk
with open("predictive_model_summary.txt", "w") as f:
    print(ols_model.summary().as_text())
    f.write(ols_model.summary().as_text())
print("OLS summary saved to predictive_model_summary.txt")


                            OLS Regression Results                            
Dep. Variable:             box_office   R-squared:                       0.548
Model:                            OLS   Adj. R-squared:                  0.546
Method:                 Least Squares   F-statistic:                     251.7
Date:                Wed, 18 Jun 2025   Prob (F-statistic):               0.00
Time:                        09:25:26   Log-Likelihood:                -95646.
No. Observations:                4799   AIC:                         1.913e+05
Df Residuals:                    4775   BIC:                         1.915e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -8.225e+